In [2]:
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
import numpy as np
import random
from sklearn.tree import DecisionTreeRegressor

## Load dataset and divide dataset in training and testset

In [3]:
df_touse, df_predict = pickle.load(open('datasets.sav','rb'))

In [4]:
df_tousebis = df_touse
predictors = df_tousebis.columns.tolist()
predictors.remove("target")
X = df_tousebis[predictors]
y = df_tousebis["target"]


X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.2)

In [5]:
X_train.head(30)
has_nan = X_train.isnull().any().any()

if has_nan:
    print("The DataFrame has NaN values.")
else:
    print("The DataFrame does not have any NaN values.")

The DataFrame has NaN values.


## Feature selection

In [6]:
from sklearn.feature_selection import VarianceThreshold

var_thr = VarianceThreshold(threshold=10)

var_thr.fit_transform(X_train.select_dtypes(include=np.number))

var_thr.get_support()

array([ True,  True,  True,  True])

## Transforming numerical variables NaNs by replacing them with median of variable to perform feature selection on numerical variables

In [11]:
# Select only the numerical columns
numeric_cols = X_train.select_dtypes(include=['int', 'float'])

# Compute the median of each numerical column
medians = numeric_cols.median()

# Replace NaN values with the median of each column
X_train[numeric_cols.columns] = numeric_cols.fillna(medians)



Y_train = Y_train.to_frame()
# Select only the numerical columns
numeric_cols = Y_train.select_dtypes(include=['int', 'float'])

# Compute the median of each numerical column
medians = numeric_cols.median()

# Replace NaN values with the median of each column
Y_train[numeric_cols.columns] = numeric_cols.fillna(medians)
Y_train = Y_train.squeeze()



## Same for Y_test

Y_test = Y_test.to_frame()
# Select only the numerical columns
numeric_cols = Y_test.select_dtypes(include=['int', 'float'])

# Compute the median of each numerical column
medians = numeric_cols.median()

# Replace NaN values with the median of each column
Y_test[numeric_cols.columns] = numeric_cols.fillna(medians)
Y_test = Y_test.squeeze()


In [15]:
has_nan = X_train.isnull().any().any()

if has_nan:
    print("The DataFrame has NaN values.")
else:
    print("The DataFrame does not have any NaN values.")

The DataFrame has NaN values.


In [16]:
# F_regression
from sklearn.feature_selection import SelectKBest, f_regression


numerical_test= SelectKBest(f_regression, k='all')
numerical_test.fit_transform(X_train.select_dtypes(include=np.number), Y_train) 

numerical_test.scores_

f_regression(X_train.select_dtypes(include=np.number), Y_train)

(array([ 608.14806985, 1407.97038666,  837.1958725 ,   16.78542599]),
 array([2.81072041e-133, 7.18936557e-303, 3.36041170e-182, 4.19358964e-005]))

## Replacing all NaNs of categorical variables to a random assignment of a category by respecting proprtions within the dataset

In [17]:
## For the training set
# Select only the categorical columns
cat_cols = X_train.select_dtypes(include=['object'])

# Iterate over the categorical columns
for col_name in cat_cols.columns:
    # Select the column
    col = X_train[col_name]
    
    # Compute the proportions of the categories
    counts = col.value_counts(normalize=True)
    
    # Replace the NaN values with random categories
    for i in col.index:
        if pd.isnull(col[i]):
            col[i] = random.choices(counts.index, counts.values)[0]
            
X_train.update(col)



## For the test set

cat_cols = X_test.select_dtypes(include=['object'])

# Iterate over the categorical columns
for col_name in cat_cols.columns:
    # Select the column
    col = X_test[col_name]
    
    # Compute the proportions of the categories
    counts = col.value_counts(normalize=True)
    
    # Replace the NaN values with random categories
    for i in col.index:
        if pd.isnull(col[i]):
            col[i] = random.choices(counts.index, counts.values)[0]
            
X_test.update(col)



/var/folders/l6/36kbn2fd34q7z3t8k7r87g8h0000gn/T/ipykernel_28868/3237520104.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col[i] = random.choices(counts.index, counts.values)[0]
/var/folders/l6/36kbn2fd34q7z3t8k7r87g8h0000gn/T/ipykernel_28868/3237520104.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col[i] = random.choices(counts.index, counts.values)[0]
/var/folders/l6/36kbn2fd34q7z3t8k7r87g8h0000gn/T/ipykernel_28868/3237520104.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [ ]:
X_train.head(30)

In [ ]:
has_nan = X_train.isnull().any().any()

if has_nan:
    print("The DataFrame has NaN values.")
else:
    print("The DataFrame does not have any NaN values.")

## OneHotencode

In [ ]:
## import not to create hot codes for this
X_train = X_train.drop(columns=['insee'])
X_test = X_test.drop(columns=['insee'])

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

## For the Train set

# Select the string columns
string_columns = X_train.select_dtypes(['object']).columns

# Create a OneHotEncoder object
encoder = OneHotEncoder()

# Fit the encoder to the string columns
encoder.fit(X_train[string_columns])

# Encode the string columns
encoded_data = encoder.transform(X_train[string_columns])

# Create a DataFrame with the encoded columns and the original index
encoded_df = pd.DataFrame(encoded_data.toarray(), index=X_train.index, columns=encoder.get_feature_names(string_columns))

# Concatenate the encoded columns with the rest of the data
X_train = pd.concat([encoded_df, X_train.drop(string_columns, axis=1)], axis=1)


## For the Test set

# Select the string columns
string_columns = X_test.select_dtypes(['object']).columns

# Create a OneHotEncoder object
encoder = OneHotEncoder()

# Fit the encoder to the string columns
encoder.fit(_test[string_columns])

# Encode the string columns
encoded_data = encoder.transform(_test[string_columns])

# Create a DataFrame with the encoded columns and the original index
encoded_df = pd.DataFrame(encoded_data.toarray(), index=_test.index, columns=encoder.get_feature_names(string_columns))

# Concatenate the encoded columns with the rest of the data
X_test = pd.concat([encoded_df, X_test.drop(string_columns, axis=1)], axis=1)

In [ ]:
has_nan = X_train.isnull().any().any()

if has_nan:
    print("The DataFrame has NaN values.")
else:
    print("The DataFrame does not have any NaN values.")

In [ ]:
X_train.head(30)

In [ ]:
X_train.shape

# Models

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error 

## DecisionTree Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import StratifiedKFold

dt_params = {'min_samples_split': [2, 5] + list(range(10, 100, 5))}

#,
#              'max_features': ['auto', 'sqrt', 'log2', None],
#              'min_impurity_decrease': [0.0, 0.1, 0.2, 0.3]}


#dt_params = {'min_samples_split': [2, 5] + list(range(10, 250,5))} 
dt = DecisionTreeRegressor(random_state=0)
cv_folds = KFold(5, shuffle=True, random_state=0)
dt_cv = GridSearchCV(dt, dt_params, cv=cv_folds, n_jobs=-1) 
dt_cv.fit(X_train, Y_train) 
print(dt_cv.best_score_)
print(dt_cv.best_params_)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

# Make predictions on the test data
predictions = dt_cv.predict(X_test)

# Compute the mean absolute error
mae = mean_absolute_error(Y_test, predictions)

# Compute the mean squared error
mse = mean_squared_error(Y_test, predictions)

# Compute the root mean squared error
rmse = math.sqrt(mse)

# Compute the R-squared score
r2 = r2_score(Y_test, predictions)

# Compute the adjusted R-squared score
n = len(Y_test)
p = X_test.shape[1]  # number of features
adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)



print("Mean absolute error: {:.2f}".format(mae))
print("Mean squared error: {:.2f}".format(mse))
print("Root mean squared error: {:.2f}".format(rmse))
print("R-squared score: {:.2f}".format(r2))
print("Adjusted R-squared score: {:.2f}".format(adj_r2))



## Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, GridSearchCV

# Define the grid of hyperparameters to search
param_grid = {'learning_rate': [0.2]}
#                                0.05, 0.01, 0.005, 0.001]}


# Create the gradient boosting model
gb = GradientBoostingRegressor(random_state=0)

# Create the K-fold cross-validation object
cv_folds = KFold(5, shuffle=True, random_state=0)

# Create the grid search object
gb_cv = GridSearchCV(gb, param_grid, cv=cv_folds, n_jobs=-1)

# Fit the grid search object to the training data
gb_cv.fit(X_train, Y_train)

print(gb_cv.best_score_)
print(gb_cv.best_params_)

In [ ]:
# Make predictions on the test data
predictions = gb_cv.predict(X_test)

# Compute the mean absolute error
mae = mean_absolute_error(Y_test, predictions)

# Compute the mean squared error
mse = mean_squared_error(Y_test, predictions)

# Compute the root mean squared error
rmse = math.sqrt(mse)

# Compute the R-squared score
r2 = r2_score(Y_test, predictions)

# Compute the adjusted R-squared score
n = len(Y_test)
p = X_test.shape[1]  # number of features
adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)



print("Mean absolute error: {:.2f}".format(mae))
print("Mean squared error: {:.2f}".format(mse))
print("Root mean squared error: {:.2f}".format(rmse))
print("R-squared score: {:.2f}".format(r2))
print("Adjusted R-squared score: {:.2f}".format(adj_r2))


## Ridge model

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, GridSearchCV

# Define the grid of hyperparameters to search
param_grid = {'alpha': [0.1, 1.0, 10.0, 100.0, 1000.0],
              'max_iter': [100, 1000, 10000, 100000]}

# Create the ridge regression model
ridge = Ridge(random_state=0)

# Create the K-fold cross-validation object
cv_folds = KFold(5, shuffle=True, random_state=0)

# Create the grid search object
ridge_cv = GridSearchCV(ridge, param_grid, cv=cv_folds, n_jobs=-1)

# Fit the grid search object to the training data
ridge_cv.fit(X_train, Y_train)

In [ ]:
print(ridge_cv.best_score_)
print(ridge_cv.best_params_)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

# Make predictions on the test data
predictions = ridge_cv.predict(X_test)

# Compute the mean absolute error
mae = mean_absolute_error(Y_test, predictions)

# Compute the mean squared error
mse = mean_squared_error(Y_test, predictions)

# Compute the root mean squared error
rmse = math.sqrt(mse)

# Compute the R-squared score
r2 = r2_score(Y_test, predictions)

# Compute the adjusted R-squared score
n = len(Y_test)
p = X_test.shape[1]  # number of features
adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)



print("Mean absolute error: {:.2f}".format(mae))
print("Mean squared error: {:.2f}".format(mse))
print("Root mean squared error: {:.2f}".format(rmse))
print("R-squared score: {:.2f}".format(r2))
print("Adjusted R-squared score: {:.2f}".format(adj_r2))



## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV

# Define the grid of hyperparameters to search
param_grid = {'n_estimators': [50, 100, 200, 300, 400, 500],
              'max_depth': [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
              'min_samples_split': [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
              'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}

# Create the random forest model
rf = RandomForestRegressor(random_state=0)

# Create the K-fold cross-validation object
cv_folds = KFold(5, shuffle=True, random_state=0)

# Create the grid search object
rf_cv = GridSearchCV(rf, param_grid, cv=cv_folds, n_jobs=-1)

# Fit the grid search object to the training data
rf_cv.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

# Make predictions on the test data
predictions = rf_cv.predict(X_test)

# Compute the mean absolute error
mae = mean_absolute_error(Y_test, predictions)

# Compute the mean squared error
mse = mean_squared_error(Y_test, predictions)

# Compute the root mean squared error
rmse = math.sqrt(mse)

# Compute the R-squared score
r2 = r2_score(Y_test, predictions)

# Compute the adjusted R-squared score
n = len(Y_test)
p = X_test.shape[1]  # number of features
adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)



print("Mean absolute error: {:.2f}".format(mae))
print("Mean squared error: {:.2f}".format(mse))
print("Root mean squared error: {:.2f}".format(rmse))
print("R-squared score: {:.2f}".format(r2))
print("Adjusted R-squared score: {:.2f}".format(adj_r2))



## LASSO